# Daily Staffing Analysis

This notebook extracts raw data from multiple per-person Excel files and generates a combined raw data table and a daily staffing summary workbook with formatted blocks.


# Dependencies


In [69]:
# ! pip install pandas xlrd openpyxl -q

In [70]:
import glob
import os
import pandas as pd
import xlrd
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
from openpyxl import Workbook
from openpyxl.styles import Font
from datetime import datetime

In [71]:
# Configuration
DATA_FOLDER = "./data/raw"
COMBINED_CSV = "./results/combined_raw_data.csv"

OUTPUT_XLSX = "./results/daily_summary_output.xlsx"
SUMMARY_SHEET = "DailyMatrix"

# Extract raw data


In [72]:
records = []
for filepath in glob.glob(os.path.join(DATA_FOLDER, "*.xls*")):
    wb = xlrd.open_workbook(filepath)
    sh = wb.sheet_by_index(0)
    individual = (
        sh.cell_value(2, 3).split(",")[0].strip()
    )  # Individual name is in D3 cell, split by comma and take first

    print(f"Processing file: {filepath} | Individual: {individual}")
    # The first 39 rows are metadata and headers, so we start from row 40

    date_cell_idx = 0
    provider_cell_idx = 6
    duration_cell_idx = 3

    for rx in range(40, sh.nrows):
        date_cell = sh.cell_value(rx, date_cell_idx)
        if date_cell in (None, ""):
            continue
        if isinstance(date_cell, str):
            try:
                dt = pd.to_datetime(date_cell).date()
            except ValueError:
                print(f"Invalid date format in row {rx}: {date_cell}")
                continue

        provider = sh.cell_value(rx, provider_cell_idx).split(",")[0].strip()
        duration = sh.cell_value(rx, duration_cell_idx)

        records.append(
            {
                "Date": dt,
                "Individual": individual,
                "Service Provider": provider,
                "Duration": duration,
            }
        )

df_raw = pd.DataFrame(records)
df_raw.to_csv(COMBINED_CSV, index=False)
print(f"Combined raw data saved to {COMBINED_CSV}")

Processing file: ./data/raw/GP.xls | Individual: George Poulston
Invalid date format in row 100: Comments/Recommendations
Processing file: ./data/raw/HR.xls | Individual: Harold Russell
Invalid date format in row 129: Comments/Recommendations
Combined raw data saved to ./results/combined_raw_data.csv


# Perform staffing analysis and save to Excel


In [ ]:
df = pd.read_csv(COMBINED_CSV, parse_dates=["Date"])
df["Date"] = df["Date"].dt.date  # ensure pure date for grouping

# --- Start a new workbook for summary ---
wb = Workbook()
ws = wb.active
ws.title = SUMMARY_SHEET
bold = Font(bold=True)
row = 1

individuals = sorted(df["Individual"].unique())

df["Duration"] = df["Duration"].apply(
    lambda x: f"{x}:00" if len(str(x).split(":")) == 2 else x
)
df["Duration"] = pd.to_timedelta(df["Duration"]).dt.total_seconds() / 3600


# --- Generate per-date blocks with formulas ---
for current_date in sorted(df["Date"].unique()):
    day_df = df[df["Date"] == current_date]

    # Date header
    ws.cell(row=row, column=1, value=current_date.strftime("%m/%d/%Y")).font = bold
    row += 1

    # Column headers
    ws.cell(row=row, column=1, value="Service Provider").font = bold
    for idx, indiv in enumerate(individuals, start=2):
        ws.cell(row=row, column=idx, value=indiv).font = bold
    total_col = 2 + len(individuals)
    ws.cell(row=row, column=total_col, value="Provider Total").font = bold
    row += 1

    # Remember where provider rows start
    provider_start = row

    # One row per service provider
    for provider in day_df["Service Provider"].unique():
        ws.cell(row=row, column=1, value=provider)

        # Write each individual’s hours
        for idx, indiv in enumerate(individuals, start=2):
            hours = day_df[
                (day_df["Service Provider"] == provider)
                & (day_df["Individual"] == indiv)
            ]["Duration"].sum()
            ws.cell(row=row, column=idx, value=hours)

        # Instead of summing in Python, insert a SUM formula
        start_letter = get_column_letter(2)
        end_letter = get_column_letter(1 + len(individuals))
        sum_range = f"{start_letter}{row}:{end_letter}{row}"
        total_cell = ws.cell(row=row, column=total_col, value=f"=SUM({sum_range})")
        total_cell.font = bold

        row += 1

    # Mark where provider rows end
    provider_end = row - 1

    # Totals per individual (SUM down each column)
    ws.cell(row=row, column=1, value="Total hours for individual").font = bold
    for idx in range(2, 2 + len(individuals)):
        col_letter = get_column_letter(idx)
        formula = f"=SUM({col_letter}{provider_start}:{col_letter}{provider_end})"
        cell = ws.cell(row=row, column=idx, value=formula)
        cell.font = bold
    row += 1

    # Remaining hours to 24h cap
    ws.cell(row=row, column=1, value="Total hrs pending in a 24hr period").font = bold
    for idx in range(2, 2 + len(individuals)):
        col_letter = get_column_letter(idx)
        above_cell = f"{col_letter}{row-1}"
        formula = f"=24 - {above_cell}"
        cell = ws.cell(row=row, column=idx, value=formula)
        cell.font = bold
    row += 2  # blank line before next date

# Save
wb.save(OUTPUT_XLSX)
print(f"Daily summary written to {OUTPUT_XLSX}")

Daily summary written to ./results/daily_summary_output.xlsx
